In [68]:
import main
bar_lis, line_lis = main.fetch_bar_lis_from_database()
print(len(bar_lis))
bar_lis


86


[[datetime.datetime(2023, 10, 29, 15, 57, 10), 1877, 1877],
 [datetime.datetime(2023, 10, 29, 15, 54, 54), 1843, 1843],
 [datetime.datetime(2023, 10, 29, 15, 54, 5), 1831, 1831],
 [datetime.datetime(2023, 10, 29, 15, 53, 9), 1817, 1817],
 [datetime.datetime(2023, 10, 29, 15, 52, 13), 1803, 1803],
 [datetime.datetime(2023, 10, 29, 15, 51, 45), 1796, 1796],
 [datetime.datetime(2023, 10, 29, 15, 51, 17), 1789, 1789],
 [datetime.datetime(2023, 10, 29, 15, 49, 45), 1766, 1766],
 [datetime.datetime(2023, 10, 29, 15, 48, 33), 1748, 1748],
 [datetime.datetime(2023, 10, 29, 15, 48, 13), 1743, 1743],
 [datetime.datetime(2023, 10, 29, 15, 46, 12), 1713, 1713],
 [datetime.datetime(2023, 10, 29, 15, 42, 28), 1657, 1619],
 [datetime.datetime(2023, 10, 29, 15, 39, 52), 1618, 1618],
 [datetime.datetime(2023, 10, 29, 15, 39, 3), 1606, 1606],
 [datetime.datetime(2023, 10, 29, 15, 37, 23), 1581, 1581],
 [datetime.datetime(2023, 10, 29, 15, 34, 23), 1536, 1536],
 [datetime.datetime(2023, 10, 29, 15, 33, 7

In [ ]:
bar_lis

In [2]:
line_lis

[78,
 174,
 175,
 243,
 244,
 355,
 356,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 561,
 562,
 563,
 564,
 580,
 581,
 582,
 583,
 696,
 697,
 760,
 761,
 762,
 763,
 781,
 782,
 783,
 788,
 789,
 790,
 791,
 792,
 871,
 872,
 873,
 876,
 877,
 976,
 977,
 978,
 994,
 995,
 996]

In [1]:
abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
rul_models_path = ['./model/RULmodel/231023_xgb_ss_df_fl.pickle','./model/RULmodel/231023_xgb_ss_df_pb.pickle','./model/RULmodel/231023_xgb_ss_df_fl.pickle']
inverse_scalers_path = ['./model/RULmodel/231023_ss_y_df_fl.pickle','./model/RULmodel/231023_ss_y_df_pb.pickle','./model/RULmodel/231023_ss_y_df_ph.pickle']
scalers_path = ['./model/RULmodel/231023_ss_x_df_fl.pickle','./model/RULmodel/231023_ss_x_df_pb.pickle','./model/RULmodel/231023_ss_x_df_ph.pickle']

In [2]:
import joblib

def load_object_with_joblib(path):
    return joblib.load(path)

def get_model_features_count(model):
    try:
        return len(model.feature_importances_)
    except AttributeError:
        return "Not available for this model"

def get_scaler_feature_count(scaler):
    try:
        return scaler.n_features_in_
    except AttributeError:
        return "Not available for this scaler"

# 모델의 컬럼 개수 확인
for path in abnormal_models_path:
    model = load_object_with_joblib(path)
    print(f"{path} model features count: {get_model_features_count(model)}")

# 스케일러의 컬럼 개수 확인
for path in scalers_path:
    scaler = load_object_with_joblib(path)
    print(f"{path} scaler features count: {get_scaler_feature_count(scaler)}")


./model/abnormal_detect/RF_model(FL).pkl model features count: 10
./model/abnormal_detect/RF_model(PB).pkl model features count: 10
./model/abnormal_detect/RF_model(PH).pkl model features count: 10
./model/RULmodel/231023_ss_x_df_fl.pickle scaler features count: 10
./model/RULmodel/231023_ss_x_df_pb.pickle scaler features count: 10
./model/RULmodel/231023_ss_x_df_ph.pickle scaler features count: 9


In [84]:
import pymysql
import pickle
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import warnings
from datetime import datetime
import joblib

# XGBoost 관련 경고 숨기기
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# 현재 시간 가져오기
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## 길이 기준 설정
avg_len = 500

def fetch_recent_logs(length=avg_len):
    """mysql db에서 최근 로그를 가져오는 함수"""
    # mysql 데이터베이스에 연결

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 가장 최근의 데이터부터 지정한 길이만큼 가져오는 SQL 쿼리
            sql = f'''SELECT IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT, ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT,
            FLOWCOOLFLOWRATE, FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
            FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
            ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER, ETCHAUX2SOURCETIMER,
            ACTUALSTEPDURATION FROM input_data_1 ORDER BY input_time DESC LIMIT {length}'''
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        connection.close()

    return results

def fetch_recent_logs_for_multi(length=avg_len):
    """mysql db에서 최근 로그를 가져오는 함수"""
    # mysql 데이터베이스에 연결

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 가장 최근의 데이터부터 지정한 길이만큼 가져오는 SQL 쿼리
            sql = f'''SELECT ACTUALROTATIONANGLE, ACTUALSTEPDURATION, ETCHBEAMCURRENT, ETCHGASCHANNEL1READBACK, 
              ETCHPBNGASREADBACK, ETCHSOURCEUSAGE, FIXTURETILTANGLE, FLOWCOOLFLOWRATE, FLOWCOOLPRESSURE, 
              IONGAUGEPRESSURE FROM input_data_1 ORDER BY input_time DESC LIMIT {length}'''
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        connection.close()

    return results

def dict_to_array(data):
    """사전 형태의 데이터를 2차원 넘파이 배열로 변환하는 함수"""
    return np.array([list(item.values()) for item in data])


def predict_with_xgb_model(data,model_path,inverse_scaler_path):
    """xgboost 모델을 사용해 예측하는 함수"""
    # 데이터 형태 변환
    transformed_data = dict_to_array(data)

    """모델 예측 전 전처리 함수"""
    # 모델 불러오기
    scaler = joblib.load('./model/rul/X_scaler.pkl')
    
    # scaler 적용
    scaled_data = scaler.transform(transformed_data)

    """xgboost 모델을 사용해 예측하는 함수"""
    # 모델 불러오기
    model =  joblib.load(model_path)

    # 예측 실행
    predictions_scaled = model.predict(scaled_data)

    #### rul inverse transfomr 적용 필요!!!!
     # 모델 불러오기
    inverse_scaler = joblib.load(inverse_scaler_path)
    
    predictions = inverse_scaler.inverse_transform(predictions_scaled.reshape(-1,1))

    return predictions[:,0]

def predict_with_xgb_multi_model(data,model_path):
    """xgboost 다중분류 모델을 사용해 예측하는 함수"""
    # 데이터 형태 변환
    transformed_data = dict_to_array(data)

    """모델 예측 전 전처리 함수"""
    # 모델 불러오기
    scaler = joblib.load('./model/abnormal_detect/StandardScaler.pkl')
    
    # scaler 적용
    scaled_data = scaler.transform(transformed_data)

    """xgboost 모델을 사용해 예측하는 함수"""
    # 모델 불러오기
    model=  joblib.load(model_path) 

    # 예측 실행
    predictions = model.predict(scaled_data)

    return predictions

def compute_moving_average(data, window_size=avg_len):
    """이동평균 계산하는 함수"""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

import numpy as np

def compute_moving_median(data, window_size):
    """이동 중앙값 계산하는 함수"""
    tmp_data = data.reshape(1,-1)
    num_data = len(tmp_data)
    medians = []

    for i in range(num_data - window_size + 1):
        window_data = tmp_data[i:i+window_size]
        medians.append(np.median(window_data))

    return np.array(medians)

########################################################################################################################################
########################################################################################################################################
########################################################################################################################################

### 데이터 밀어 넣기

def insert_single_data(connection, single_data):
    try:
        with connection.cursor() as cursor:
            # 현재 시간 가져오기
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO input_data_1 (time, Tool, stage, Lot, runnum, recipe, recipe_step,
       IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT,
       ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT, FLOWCOOLFLOWRATE,
       FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
       FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
       FIXTURESHUTTERPOSITION, ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER,
       ETCHAUX2SOURCETIMER, ACTUALSTEPDURATION, input_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "{current_time}")'''
            cursor.execute(sql, single_data)
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()
    return current_time

def insert_single_rul_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO rul_1(rul_fl, rul_pb, rul_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()

def insert_single_multi_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO multi_1(multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()




def main():
    # 데이터베이스 연결 설정

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    # CSV 파일 읽기
    df = pd.read_csv("./model_data_input/test.csv")

    # DataFrame에서 튜플 리스트로 데이터 변환
    data_tuples = list(df.itertuples(index=False, name=None))

    abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
    rul_models_path = ['./model/rul/xgb_model_for_fl.pkl','./model/rul/xgb_model_for_pb.pkl','./model/rul/xgb_model_for_ph.pkl']
    inverse_scalers_path = ['./model/rul/y_scaler_for_fl.pkl','./model/rul/y_scaler_for_pb.pkl','./model/rul/y_scaler_for_ph.pkl']

    ## 데이터를 한줄 씩 밀어넣으면서 진행하는 방식

    for single_data in data_tuples:

        try:
            # 데이터 삽입
            current_time = insert_single_data(connection, single_data)

            data = fetch_recent_logs(length=avg_len)
            data_for_multi = fetch_recent_logs_for_multi(length=avg_len)

            rul_insert = []
            multi_insert = []

            for i in range(len(abnormal_models_path)):
            
                # 가져온 데이터를 기반으로 예측하기
                predictions = predict_with_xgb_model(data,rul_models_path[i],inverse_scalers_path[i])
                predictions_for_multi = predict_with_xgb_multi_model(data_for_multi,abnormal_models_path[i])
                
                # 이동평균 계산하기
                window_size = min(avg_len, len(predictions))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
                moving_avg = compute_moving_average(predictions, window_size=window_size)
                window_size_for_multi = min(avg_len, len(predictions_for_multi))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
                moving_avg_for_multi = compute_moving_average(predictions_for_multi, window_size=window_size_for_multi)
                
                rul_insert.append(moving_avg[0])
                print(6)
                multi_insert.append(predictions_for_multi[0])
                print(7)
                
            # pred 데이터 삽입
            insert_single_rul_data(connection, rul_insert, current_time)
            print(8)
            insert_single_multi_data(connection, multi_insert, current_time)
            print(9)

            time.sleep(4)  # 4초 대기

        except Exception as e:
            print(f"Error: {e}")
    
    # 연결 종료
    connection.close()    

main()


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


KeyboardInterrupt: 

In [75]:

# CSV 파일 읽기
df = pd.read_csv("./model_data_input/test.csv")

# DataFrame에서 튜플 리스트로 데이터 변환
data_tuples = list(df.itertuples(index=False, name=None))

abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
rul_models_path = ['./model/rul/xgb_model_for_fl.pkl','./model/rul/xgb_model_for_pb.pkl','./model/rul/xgb_model_for_ph.pkl']
inverse_scalers_path = ['./model/rul/y_scaler_for_fl.pkl','./model/rul/y_scaler_for_pb.pkl','./model/rul/y_scaler_for_ph.pkl']

## 데이터를 한줄 씩 밀어넣으면서 진행하는 방식

In [11]:
connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

NameError: name 'pymysql' is not defined

In [12]:
data = fetch_recent_logs(length=avg_len)
data

NameError: name 'fetch_recent_logs' is not defined

In [78]:
predictions = predict_with_xgb_model(data,rul_models_path[0],inverse_scalers_path[0])
predictions

c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[2169449. ],
       [2169449. ],
       [2169449. ],
       [2169449. ],
       [1580659.8],
       [1541943.1],
       [1875674.9],
       [1819576.9],
       [1396611.4],
       [1735063.5],
       [1735063.5],
       [1733304.1],
       [2054692.5],
       [2056451.9],
       [1735063.5],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1875674.9],
       [1733304.1],
       [2054692.5],
       [1497973.1],
       [1258518.9],
       [1344791.4],
       [1734558.6],
       [1489876.6],
       [1624971.2],
       [1562869.8],
       [1467559.1],
       [1693362.1],
       [1693362.1],
       [1692258.5],
       [1808169.8],
       [1861418.4],
       [1692258.5],
       [1692258.5],
       [1861418.4],
       [1861418.4],
       [1808169.8],
       [1778530.9],
       [1894442.5],
       [1894442.5],
       [1809929.1],
       [1694017.9],
       [1723802.4],
       [1839713.6],
       [1839713.6],
       [1892962.2],


In [79]:
window_size = min(avg_len, len(predictions))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
window_size

398

In [80]:
predictions[:,0]

array([2169449. , 2169449. , 2169449. , 2169449. , 1580659.8, 1541943.1,
       1875674.9, 1819576.9, 1396611.4, 1735063.5, 1735063.5, 1733304.1,
       2054692.5, 2056451.9, 1735063.5, 1733304.1, 1733304.1, 1733304.1,
       1733304.1, 1733304.1, 1875674.9, 1733304.1, 2054692.5, 1497973.1,
       1258518.9, 1344791.4, 1734558.6, 1489876.6, 1624971.2, 1562869.8,
       1467559.1, 1693362.1, 1693362.1, 1692258.5, 1808169.8, 1861418.4,
       1692258.5, 1692258.5, 1861418.4, 1861418.4, 1808169.8, 1778530.9,
       1894442.5, 1894442.5, 1809929.1, 1694017.9, 1723802.4, 1839713.6,
       1839713.6, 1892962.2, 1597665. , 1809259.9, 1978703. , 1936332. ,
       1978703. , 1978703. , 1995485.5, 1995485.5, 1775048.4, 1978703. ,
       1978703. , 1978703. , 1978703. , 1978703. , 2114300.5, 2049412.1,
       1978703. , 1978703. , 2114300.5, 1910645.6, 2114300.5, 1936332. ,
       1954776. , 2116059.8, 2116059.8, 1980462.4, 1980462.4, 1950332.5,
       1950332.5, 1927738.2, 1760793.6, 1469662.6, 

In [81]:
moving_avg = compute_moving_average(predictions[:,0], window_size=window_size)
moving_avg

array([1968009.60081658])

In [26]:
current_time = insert_single_data(connection, data_tuples[0])
current_time

'2023-10-24 12:36:08'

In [27]:
data_for_multi = fetch_recent_logs_for_multi(length=avg_len)
data_for_multi

[{'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'FIXTURETILTANGLE': 2.027847975076401,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'IONGAUGEPRESSURE': -0.0794283467426016},
 {'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'FIXTURETILTANGLE': 2.027847975076401,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'IONGAUGEPRESSURE': -0.0794283467426016},
 {'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  

In [28]:
predictions_for_multi = predict_with_xgb_multi_model(data_for_multi,abnormal_models_path[0])
predictions_for_multi

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [29]:
multi_insert = []

In [30]:
multi_insert.append(predictions_for_multi[0])

In [31]:
multi_insert

[0]

In [17]:
len(data_for_multi[0].values())

10

In [19]:
import mysql.connector

In [20]:
db = mysql.connector.connect(
    host="limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com",
    user="oneday",
    password="1234",
    database="j6database",
)


# 커서 생성
cursor = db.cursor()


In [21]:
cursor.execute(f"""SELECT temp.*, (row_index - LAG(row_index) OVER (ORDER BY row_index)) as diff
                FROM (SELECT multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                    FROM multi_1) AS temp
                WHERE (multi_pred_fl = 1) or (multi_pred_pb = 1) or (multi_pred_ph = 1);""")
existing_user = cursor.fetchall()
existing_user

[(0, 0, 1, datetime.datetime(2023, 10, 27, 23, 1, 43), 14, None),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 3, 23), 39, 25),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 4, 19), 53, 14),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 4, 23), 54, 1),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 5, 47), 75, 21),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 7, 52), 106, 31),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 9, 36), 132, 26),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 11, 20), 158, 26),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 11, 44), 164, 6),
 (1, 1, 0, datetime.datetime(2023, 10, 27, 23, 12, 20), 173, 9),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 14, 36), 207, 34),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 17, 1), 243, 36),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 18, 49), 270, 27),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 20, 33), 296, 26),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 21, 21), 308, 12),
 (0, 1, 0, datetime.date

In [42]:
(existing_user[2][-1]) != 1

True

In [23]:
bar_lis = [[existing_user[0][3], existing_user[0][4]]]
bar_lis

[[datetime.datetime(2023, 10, 27, 23, 1, 43), 14]]

In [24]:
for i in range(1, len(existing_user)):
    if existing_user[i][-1] != 1 :
        bar_lis[len(bar_lis)-1].append(existing_user[i-1][4])
        bar_lis.append([existing_user[i][3], existing_user[i][4]])
bar_lis[-1].append(existing_user[-1][4])

In [25]:
bar_lis

[[datetime.datetime(2023, 10, 27, 23, 1, 43), 14, 14],
 [datetime.datetime(2023, 10, 27, 23, 3, 23), 39, 39],
 [datetime.datetime(2023, 10, 27, 23, 4, 19), 53, 54],
 [datetime.datetime(2023, 10, 27, 23, 5, 47), 75, 75],
 [datetime.datetime(2023, 10, 27, 23, 7, 52), 106, 106],
 [datetime.datetime(2023, 10, 27, 23, 9, 36), 132, 132],
 [datetime.datetime(2023, 10, 27, 23, 11, 20), 158, 158],
 [datetime.datetime(2023, 10, 27, 23, 11, 44), 164, 164],
 [datetime.datetime(2023, 10, 27, 23, 12, 20), 173, 173],
 [datetime.datetime(2023, 10, 27, 23, 14, 36), 207, 207],
 [datetime.datetime(2023, 10, 27, 23, 17, 1), 243, 243],
 [datetime.datetime(2023, 10, 27, 23, 18, 49), 270, 270],
 [datetime.datetime(2023, 10, 27, 23, 20, 33), 296, 296],
 [datetime.datetime(2023, 10, 27, 23, 21, 21), 308, 308],
 [datetime.datetime(2023, 10, 27, 23, 21, 33), 311, 311],
 [datetime.datetime(2023, 10, 27, 23, 22, 17), 322, 322],
 [datetime.datetime(2023, 10, 27, 23, 22, 45), 329, 329],
 [datetime.datetime(2023, 10,

In [20]:
cursor.execute(f"""SELECT row_index
                        FROM (SELECT rul_fl, rul_pb, rul_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                        FROM rul_1) AS temp
                        WHERE (rul_fl < 10) or (rul_pb < 10) or (rul_ph < 10);""")
existing_user = cursor.fetchall()


In [21]:
line_lis = [val[0] for val in existing_user]
line_lis

[287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348]

In [17]:
line_lis = []

In [29]:
cursor.execute(f"""SELECT distinct DATE_FORMAT(input_time, '%H:%i:%s'), ACTUALROTATIONANGLE, FIXTURETILTANGLE,
                        ETCHBEAMCURRENT,IONGAUGEPRESSURE,
                        ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
                        ACTUALSTEPDURATION, ETCHSOURCEUSAGE,
                        FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE
                    FROM input_data_1;""")
existing_user = cursor.fetchall()

In [34]:
existing_user[0]

('11:05:11',
 -0.1224370708389037,
 2.027847975076401,
 -0.9595093781327426,
 -0.0794283467426016,
 -1.751661148558742,
 -2.783162253978537,
 4.03778468798202,
 2.3953704185454097,
 -2.842906518897486,
 -2.3328068957350747)

In [39]:
for i in range(1,11):
    print(i)

1
2
3
4
5
6
7
8
9
10


In [38]:
for each in existing_user:
    if len(each) != 11:
        print(each)

# len(existing_user[0])

In [31]:
colnames = cursor.description  # 변수정보
cols = [[i, colnames[i][0], colnames[i+1][0]] for i in range(1, len(colnames), 2)]  # 변수명
    

In [33]:
existing_user[0],existing_user[1]

(('11:05:11',
  -0.1224370708389037,
  2.027847975076401,
  -0.9595093781327426,
  -0.0794283467426016,
  -1.751661148558742,
  -2.783162253978537,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747),
 ('11:05:16',
  -0.1224370708389037,
  2.027847975076401,
  -0.9596512376809438,
  -0.0794187434958652,
  -1.7439048724585284,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747))

In [40]:
# 각 val의 길이를 출력하여 확인
for val in existing_user:
    print(len(val))

# i의 최대 값이 val의 길이보다 작은지 확인
for val in existing_user:
    if i >= len(val):
        print(f"Index {i} is out of range for val with length {len(val)}")


11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
1

In [47]:
# i의 최대 값이 val의 길이보다 작은지 확인
for val in existing_user:
    if 11 >= len(val):
        print(f"Index 10 is out of range for val with length {len(val)}")


Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with le

In [42]:
print(f"i value: {i}")
for val in existing_user:
    print(f"Tuple length: {len(val)}")

i value: 10
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tu

In [43]:
reference_length = len(existing_user[0])
for val in existing_user:
    if len(val) != reference_length:
        print(f"Tuple with different length found: {len(val)}")


In [45]:
existing_user

[('11:05:11',
  -0.1224370708389037,
  2.027847975076401,
  -0.9595093781327426,
  -0.0794283467426016,
  -1.751661148558742,
  -2.783162253978537,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747),
 ('11:05:16',
  -0.1224370708389037,
  2.027847975076401,
  -0.9596512376809438,
  -0.0794187434958652,
  -1.7439048724585284,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747),
 ('11:05:20',
  -0.1224370708389037,
  2.027847975076401,
  -0.960005916785792,
  -0.079408642268514,
  -1.7439048724585284,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.841849516088073,
  -2.3328068957350747),
 ('11:05:24',
  -0.1224370708389037,
  2.027847975076401,
  -0.9592965888104408,
  -0.0794198349017626,
  -1.7471575247996358,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.841849516088073,
  -2.3328068957350747),
 ('11:05:28',
  -0.1224370708389037,
  2.02784797507

In [44]:
if not existing_user:
    print("existing_user is empty!")
    return


SyntaxError: 'return' outside function (2388791911.py, line 3)

In [48]:
def fetch_bar_lis_from_database(n=1):
    cursor.execute(f"""SELECT distinct DATE_FORMAT(input_time, '%H:%i:%s'), ACTUALROTATIONANGLE, FIXTURETILTANGLE,
                        ETCHBEAMCURRENT,IONGAUGEPRESSURE,
                        ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
                        ACTUALSTEPDURATION, ETCHSOURCEUSAGE,
                        FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE
                    FROM input_data_{n};""")
    existing_user = cursor.fetchall()
    
    colnames = cursor.description  # 변수정보
    cols = [[i, colnames[i][0], colnames[i+1][0]] for i in range(1, len(colnames), 2)]  # 변수명
    
    alram_dic = {}
    for i in range(1, len(existing_user[0])):
        key = 'alram{}'.format(i)
        alram_dic[key] = [{'time': val[0], 'col': val[i]} for val in existing_user]

        try:
            cursor.execute(f"""SELECT row_index
                            FROM (SELECT rul_fl, rul_pb, rul_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                            FROM rul_1) AS temp
                            WHERE (rul_fl < 10) or (rul_pb < 10) or (rul_ph < 10);""")
            existing_user = cursor.fetchall()
            line_lis = [val[0] for val in existing_user]
        
            cursor.execute(f"""SELECT temp.*, (row_index - LAG(row_index) OVER (ORDER BY row_index)) as diff
                            FROM (SELECT multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                                FROM multi_{n}) AS temp
                            WHERE (multi_pred_fl = 1) or (multi_pred_pb = 1) or (multi_pred_ph = 1);""")
            existing_user = cursor.fetchall()
        
            bar_lis = [[existing_user[0][3], existing_user[0][4]]]
            cnt = 0
            for i in range(1, len(existing_user)):
                if existing_user[i][-1] != 1:
                    bar_lis[len(bar_lis) - 1].append(existing_user[i - 1][4])
                    bar_lis.append([existing_user[i][3], existing_user[i][4]])
            bar_lis[-1].append(existing_user[-1][4])

        except:
            line_lis = None
            bar_lis = [[None, 0, 0],[None, 0, 0],[None, 0, 0],[None, 0, 0]]
    
        return bar_lis

In [51]:
fetch_bar_lis_from_database()

[[datetime.datetime(2023, 10, 27, 6, 44, 45), 4513, 4517]]

In [56]:
cursor.execute(f"""SELECT temp.*, (row_index - LAG(row_index) OVER (ORDER BY row_index)) as diff
                        FROM (SELECT multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                            FROM multi_{1}) AS temp
                        WHERE (multi_pred_fl = 1) or (multi_pred_pb = 1) or (multi_pred_ph = 1);""")
existing_user = cursor.fetchall()
# existing_user[0][-1] = existing_user[0][-2]
existing_user

[(0, 0, 1, datetime.datetime(2023, 10, 27, 23, 1, 43), 14, None),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 3, 23), 39, 25),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 4, 19), 53, 14),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 4, 23), 54, 1),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 5, 47), 75, 21),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 7, 52), 106, 31),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 9, 36), 132, 26),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 11, 20), 158, 26),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 11, 44), 164, 6),
 (1, 1, 0, datetime.datetime(2023, 10, 27, 23, 12, 20), 173, 9),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 14, 36), 207, 34),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 17, 1), 243, 36),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 18, 49), 270, 27),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 20, 33), 296, 26),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 21, 21), 308, 12),
 (0, 1, 0, datetime.date

In [53]:
bar_lis = [[existing_user[-1][3], existing_user[-1][4]]]
bar_lis

[[datetime.datetime(2023, 10, 29, 15, 58, 2), 1890]]

In [58]:
for i in range(len(existing_user) - 2, -1, -1):
    if existing_user[i][-1] != 1:
    # if existing_user[i][-1] != 1 :
        bar_lis[len(bar_lis) - 1].append(existing_user[i + 1][4])
        bar_lis.append([existing_user[i][3], existing_user[i][4]])
bar_lis


[[datetime.datetime(2023, 10, 29, 15, 58, 2), 1890, 1890],
 [datetime.datetime(2023, 10, 29, 15, 57, 10), 1877, 1877],
 [datetime.datetime(2023, 10, 29, 15, 54, 54), 1843, 1843],
 [datetime.datetime(2023, 10, 29, 15, 54, 5), 1831, 1831],
 [datetime.datetime(2023, 10, 29, 15, 53, 9), 1817, 1817],
 [datetime.datetime(2023, 10, 29, 15, 52, 13), 1803, 1803],
 [datetime.datetime(2023, 10, 29, 15, 51, 45), 1796, 1796],
 [datetime.datetime(2023, 10, 29, 15, 51, 17), 1789, 1789],
 [datetime.datetime(2023, 10, 29, 15, 49, 45), 1766, 1766],
 [datetime.datetime(2023, 10, 29, 15, 48, 33), 1748, 1748],
 [datetime.datetime(2023, 10, 29, 15, 48, 13), 1743, 1743],
 [datetime.datetime(2023, 10, 29, 15, 46, 12), 1713, 1713],
 [datetime.datetime(2023, 10, 29, 15, 42, 28), 1657, 1657],
 [datetime.datetime(2023, 10, 29, 15, 39, 56), 1619, 1619],
 [datetime.datetime(2023, 10, 29, 15, 39, 52), 1618, 1618],
 [datetime.datetime(2023, 10, 29, 15, 39, 3), 1606, 1606],
 [datetime.datetime(2023, 10, 29, 15, 37, 23

In [71]:
bar_lis = []

# 마지막 열 값이 >1 인 데이터와 None 값을 무시
for idx, row in enumerate(existing_user[:-1]):
    if row[-1] is not None and row[-1] > 1:
        current_datetime = row[3]
        current_value = row[-2]
        next_value = existing_user[idx + 1][-2]
        bar_lis.append((current_datetime, current_value, next_value))

# 전체 순서 역으로 정렬
bar_lis = sorted(bar_lis, key=lambda x: x[0], reverse=True)

In [72]:
filtered_data

[(datetime.datetime(2023, 10, 29, 15, 57, 10), 1877, 1890),
 (datetime.datetime(2023, 10, 29, 15, 54, 54), 1843, 1877),
 (datetime.datetime(2023, 10, 29, 15, 54, 5), 1831, 1843),
 (datetime.datetime(2023, 10, 29, 15, 53, 9), 1817, 1831),
 (datetime.datetime(2023, 10, 29, 15, 52, 13), 1803, 1817),
 (datetime.datetime(2023, 10, 29, 15, 51, 45), 1796, 1803),
 (datetime.datetime(2023, 10, 29, 15, 51, 17), 1789, 1796),
 (datetime.datetime(2023, 10, 29, 15, 49, 45), 1766, 1789),
 (datetime.datetime(2023, 10, 29, 15, 48, 33), 1748, 1766),
 (datetime.datetime(2023, 10, 29, 15, 48, 13), 1743, 1748),
 (datetime.datetime(2023, 10, 29, 15, 46, 12), 1713, 1743),
 (datetime.datetime(2023, 10, 29, 15, 42, 28), 1657, 1713),
 (datetime.datetime(2023, 10, 29, 15, 39, 52), 1618, 1619),
 (datetime.datetime(2023, 10, 29, 15, 39, 3), 1606, 1618),
 (datetime.datetime(2023, 10, 29, 15, 37, 23), 1581, 1606),
 (datetime.datetime(2023, 10, 29, 15, 34, 23), 1536, 1581),
 (datetime.datetime(2023, 10, 29, 15, 33, 7